In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
import psycopg2
import re

In [9]:
#스크롤 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,6000);")
    time.sleep(2)

# 해시태그 함수
def get_hashtag(outline_test) :
    # hash_tag가 안달려있는 영상이 있기때문에 오류 방지
    try:
        hash_tag = " ".join(re.findall("#[가-힣]{1,}", outline_test))
        #print(hash_tag)
        hashtag_list.append(hash_tag)
            
    except Exception as e:
        hash_tag = ""
        #print(hash_tag)
        hashtag_list.append(hash_tag)

#select문 만들기
def select_command(table) :
    command = 'select * from ' + table
    return command

#insert문 만들기
def insert_command(table) :
    command = 'insert into ' + table + ' (title, date, url, hashtag) values (%s, %s, %s, %s);'
    return command

뷔스티에 원피스, 롱패딩, 숏패딩, 롱코트, 숏코트, 잠옷, 홈웨어, 트위드자켓, 블레이저 자켓, 싱글 자켓, 크롭 자켓, 레더 자켓, 루즈핏, 테일러드 자켓, 린넨자켓, 반팔자켓, 라이더자켓, 오버핏자켓, 청자켓, 니트가디건, 볼레로, 아가일, 크롭가디건, 반팔가디건', '앙고라', '바람막이', '아노락', '윈드브레이커', '바시티자켓', '바시티점퍼', '야구점퍼', 후드집업, 투웨이집업, 여름아우터, 간절기아우터, 겨울아우터, 기모맨투맨, 니트맨투맨, 크롭 맨투맨', '남자 맨투맨', '반집업', 후드티, 남자후드, 꾸안꾸, 플리스, 반팔티, 크롭티, 블라우스, 와이셔츠, 셔츠, 체크셔츠, 타이다이, 옥스퍼드셔츠, 남방, 슬리브티, 니트, 남자니트, 조끼, '봄슬랙스', '여름슬랙스', '가을슬랙스', '겨울슬랙스', '간절기슬랙스', '데님팬츠', '연청', '진청', '중청', '흑청', '그레이진', '와이드팬츠', '밴딩바지', '스키니진', '린넨팬츠', '반바지', '하이웨스트', '로우웨스트', '부츠컷', '치노팬츠', '트레이닝복', '트레이닝바지', '조거팬츠', '카고바지', '일자팬츠', '냉장고바지', '멜빵바지', '오버롤팬츠', '찢청', 

keyword_list = ['미니스커트', '미디스커트', '롱스커트', '플레어스커트', 'A라인 스커트', 'H라인 스커트', '청치마', '테니스스커트', '프릴스커트', '레더 스커트', '랩스커트', '데님스커트', '플리츠스커트', ]

In [14]:
keyword_list = ['린넨원피스', '카라원피스', '랩원피스', '니트원피스', '후드원피스', '코듀로이', '반팔원피스', '점프슈트', '봄룩북', '봄옷쇼핑', '봄코디', '봄옷하울', '여름룩북', '여름옷쇼핑', '여름코디', '여름하울', '가을룩북', '가을옷쇼핑', '가을코디', '가을하울']
#'미니원피스', '미디원피스', '롱원피스', '나시원피스', '코르셋', '셔츠원피스', '벨벳원피스', '꽃원피스', '골지원피스', '골지티', '쉬폰원피스', '쉬폰블라우스', 

In [15]:
warnings.filterwarnings(action='ignore')

driver = webdriver.Chrome('C:\chromedriver.exe')

for keyword in keyword_list :

    # 키워드를 검색한 유튜브 주소 접속
    url = 'https://www.youtube.com/results?search_query={}'.format(keyword)
    driver.get(url)

    #사이트 최대화
    driver.maximize_window()

    # 필터 클릭
    driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/div/ytd-toggle-button-renderer/a/tp-yt-paper-button').click()
    time.sleep(1)

    # 구분 -> 동영상 클릭
    driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[2]/ytd-search-filter-renderer[1]/a/div/yt-formatted-string').click()

    # 업로드 날짜 -> 올해 클릭
    driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[5]/a').click()


    # 스크롤 다운 50번 실행
    body = driver.find_element_by_tag_name('body')
    body.send_keys(Keys.PAGE_DOWN)

    for i in range(1,50):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    name = soup.select('a#video-title')
    video_url = soup.select('a#video-title')

    video_num_list = [] #영상번호
    name_list = [] #영상제목
    url_list = [] #영상주소

    for i in range(len(name)):
        video_num_list.append(i+1)
        name_list.append(name[i].text.strip())

    for i in video_url:
        url_list.append('{}{}'.format('https://www.youtube.com',i.get('href')))

    print(keyword, '영상 개수: ', len(name_list))

    # 업로드일 & 해시태그 가져오기
    day_list = [] #업로드일
    hashtag_list = [] #해시태그

    for i in range(0,len(name_list)):

        print(i+1, end=' ')
        
        if url_list[i].split('/')[3] == 'shorts':  #쇼츠

            html_source = driver.page_source
            soup = BeautifulSoup(html_source, 'html.parser')
            
            driver.get(url_list[i])

            time.sleep(2)

            #더보기 클릭
            driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-shorts/div[1]/ytd-reel-video-renderer[1]/div[2]/ytd-reel-player-overlay-renderer/div[2]/div[1]/ytd-menu-renderer/yt-icon-button/button/yt-icon').click()
            time.sleep(2)

            #설명 클릭
            driver.find_element_by_xpath('/html/body/ytd-app/ytd-popup-container/tp-yt-iron-dropdown/div/ytd-menu-popup-renderer/tp-yt-paper-listbox/ytd-menu-service-item-renderer/tp-yt-paper-item').click()
            time.sleep(2)

            #업로드일 정보 가져오기
            day = driver.find_element_by_id("publish-time").text.split(':')[1].strip()
            day_list.append(day)

            time.sleep(2) 

            # 해시태그 가져오기
            outline_test = driver.find_element_by_class_name("style-scope ytd-reel-description-sheet-renderer").text
            get_hashtag(outline_test)

        else :

            driver.get(url_list[i])
                
            time.sleep(2)    

            html_source = driver.page_source
            soup = BeautifulSoup(html_source, 'html.parser')

            time.sleep(2)  

            #업로드일 정보 가져오기
            day = soup.select('#formatted-snippet-text > span:nth-child(3)')[0].text
            #day = driver.find_element_by_id("info-strings").text.split('\n')[0]
            day_list.append(day)

            if day == "" :
                print('error in getting day')
                break

            time.sleep(2) 

            # 해시태그 가져오기
            outline_test = soup.find("yt-formatted-string", {"class":"content style-scope ytd-video-secondary-info-renderer"}).text    
            get_hashtag(outline_test)

    print('끝')

    day_list2 = [] #전처리 후 업로드일

    #업로드일 날짜형식으로 통일화
    for day in day_list : 
        day = day.replace('.', '')
        day = day.replace('최초 공개: ', '')
        day = day.replace('실시간 스트리밍 시작일: ', '')
        day = day.replace(' ', '-')
        day_list2.append(day)

    #df화
    youtubeDic = {
        '제목': name_list,
        '업로드일': day_list2,
        '주소': url_list,
        '해시태그' : hashtag_list
    }

    youtubeDf = pd.DataFrame(youtubeDic)

    # datetime 타입으로 변환
    youtubeDf['업로드일'] = pd.to_datetime(youtubeDf['업로드일'])

    # 중복값제거
    youtubeDf = youtubeDf.drop_duplicates(['주소'], keep='first')

    print(youtubeDf.tail())


    #postgresql 접속
    conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
    conn = psycopg2.connect(conn_string)
    cur = conn.cursor()

    table = 'yt_clothes'

    #table에 데이터 넣기
    for i in range(len(youtubeDf)) :
        cur.execute(insert_command(table), (youtubeDf.iloc[i][0], youtubeDf.iloc[i][1], youtubeDf.iloc[i][2], youtubeDf.iloc[i][3]))
        conn.commit()

    #db 연결 종료
    conn.cursor().close()
    conn.close()
    

린넨원피스 영상 개수:  138
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 끝
                                               제목       업로드일  \
131                               제뉴리스 린넨 원피스 앞치마 2022-04-12   
132                  강아지 여름 원피스 언박싱 [동동이네 린넨 원피스] 2021-07-13   
133         #시원하고고급스러운 여름최강의상#린넨원피스 #핸드메이드의상#수입린넨 2021-07-26   
135  린넨통바지,린넨플레어스커트 코디법/153cm 통바지가 키를 더크게 보이게해요~~ 2022-04-09   
136                    #164 여름 원피스, 너로 정했어! 리넨 특집 2021-07-05   

                                              주소  \
131  https://www.youtube.com/watch?v=vNlOnMge4qw   
132  https://www.youtube.com/watch?v=3DnXJ8umoc

ParserError: Unknown string format: 최초-공개일:-2022-6-2

In [16]:
day_list



['2021. 10. 4.',
 '2021. 10. 19.',
 '2021. 10. 22.',
 '최초 공개일: 2022. 6. 2.',
 '2021. 9. 24.',
 '2021. 10. 22.',
 '2021. 9. 17.',
 '2021. 9. 26.',
 '2021. 9. 25.',
 '2021. 9. 30.',
 '2021. 9. 29.',
 '2021. 10. 6.',
 '2021. 10. 29.',
 '2021. 10. 24.',
 '2021. 10. 31.',
 '2021. 10. 7.',
 '2022. 5. 28.',
 '2021. 9. 26.',
 '2021. 9. 19.',
 '2021. 9. 9.',
 '2021. 10. 1.',
 '최초 공개: 2021. 9. 25.',
 '2021. 9. 7.',
 '2021. 10. 3.',
 '2021. 8. 31.',
 '2021. 9. 21.',
 '2021. 9. 2.',
 '2021. 10. 26.',
 '2021. 9. 23.',
 '2021. 9. 2.',
 '2021. 10. 4.',
 '2021. 9. 30.',
 '2021. 10. 1.',
 '2021. 9. 23.',
 '2021. 10. 24.',
 '2021. 8. 31.',
 '2021. 10. 22.',
 '2021. 9. 15.',
 '2021. 10. 6.',
 '2021. 9. 9.',
 '2021. 9. 27.',
 '2021. 9. 28.',
 '2021. 8. 28.',
 '2021. 10. 19.',
 '2021. 11. 5.',
 '2021. 8. 19.',
 '2021. 10. 31.',
 '2021. 8. 4.',
 '2021. 8. 23.',
 '2021. 11. 2.',
 '2021. 11. 1.',
 '2021. 10. 20.',
 '2021. 9. 13.',
 '2021. 9. 24.',
 '2021. 10. 22.',
 '2021. 9. 19.',
 '2021. 9. 13.',
 '2021. 8.

In [17]:
day_list2 = [] #전처리 후 업로드일

    #업로드일 날짜형식으로 통일화
for day in day_list : 
    day = day.replace('.', '')
    day = day.replace('최초 공개: ', '')
    day = day.replace('최초 공개일: ', '')
    day = day.replace('실시간 스트리밍 시작일: ', '')
    day = day.replace(' ', '-')
    day_list2.append(day)

    #df화
youtubeDic = {
    '제목': name_list,
    '업로드일': day_list2,
    '주소': url_list,
    '해시태그' : hashtag_list
}

youtubeDf = pd.DataFrame(youtubeDic)

    # datetime 타입으로 변환
youtubeDf['업로드일'] = pd.to_datetime(youtubeDf['업로드일'])

    # 중복값제거
youtubeDf = youtubeDf.drop_duplicates(['주소'], keep='first')

print(youtubeDf.tail())


    #postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

table = 'yt_clothes'

    #table에 데이터 넣기
for i in range(len(youtubeDf)) :
    cur.execute(insert_command(table), (youtubeDf.iloc[i][0], youtubeDf.iloc[i][1], youtubeDf.iloc[i][2], youtubeDf.iloc[i][3]))
    conn.commit()

    #db 연결 종료
conn.cursor().close()
conn.close()

                                                    제목       업로드일  \
140               7가지 가을 룩북 영상🍁ㅣFall lookbook , 7looks 2021-10-04   
141                      가을맞이 룩북 ✨ 캐주얼부터 여리여리까지 꾸안꾸 착장 2021-09-05   
142  [가을룩북] 평범한 여고딩의 7가지 데일리 룩북 • OOTD • 고등학생 코디 • ... 2021-10-14   
143  기본템 활용하기 뽀짝포인트 가을 룩북 | 마리떼 더오픈프로덕트 젠틀몬스터 아마레또 혜림무 2021-10-15   
144  🍂가을룩북2탄 호다닥🧥 (feat. 아모멘토 가디건, CIDER, 브라운레더자켓, ... 2021-11-18   

                                              주소  \
140  https://www.youtube.com/watch?v=-xmEOCffHtA   
141  https://www.youtube.com/watch?v=Xvvfcgu6pAA   
142  https://www.youtube.com/watch?v=bhqAmUAW66g   
143  https://www.youtube.com/watch?v=OymZ_m1ZIRE   
144  https://www.youtube.com/watch?v=plyIsMYFxLU   

                                                  해시태그  
140  #가을 #룩북 #가을룩북 #남자패션 #가을패션 #남자룩북 #빈티지 #보테가베네타 #...  
141                             #가을코디여자 #가을꾸안꾸룩 #데일리룩북  
142  #가을룩북 #가을데일리룩 #고등학생 #학생룩북 #브이로그 #데일리룩북 #오늘의코디 ...  
143             #마리떼 #노멜 #더오픈프로덕트 #젠틀몬스터 #폴로

,제목,업로드일,주소,해시태그
147,다양한 색상의 조거팬츠 얻는 방법!,2022-06-01,https://www.youtube.com/watch?v=GVkL-JXJASM,
148,양기모 츄리닝 세트 조거 팬츠 착샷 리리베리,2021-12-01,https://www.youtube.com/shorts/SSasZgHy3dw,
149,카모 카고 밀리터리 스판 밴딩 건빵 조거팬츠,2021-11-30,https://www.youtube.com/watch?v=Uc3H183L2r8,
150,"맨투맨, 후드티, 조거팬츠, 패딩 활용법/ 캐주얼코디, 꾸안꾸 데일리룩~ 캠핑룩까지...",2021-11-08,https://www.youtube.com/watch?v=Ags38QP4x4E,#조거팬츠코디 #추리닝코디 #캠핑룩
151,플스 후리스 밴딩 조거 팬츠,2022-01-18,https://www.youtube.com/shorts/x0WRi10iuak,


In [477]:
#varchar(500) type에 안들어가면 실행
b = 0

for i in range(len(hashtag_list)) :
    a = len(hashtag_list[i])

    if a > b :
        b = a
        print(i, b)

print(hashtag_list[112][:201])

0 24
1 30
11 100
37 135
74 176
112 1512
#크롭탑 #브라탑 #가보시 #진주백 #부츠 #프라다 #원형백 #블랙 #밍크 #밍크퍼 #부츠컷 #청바지 #부츠컷청바지 #퍼자켓 #가디건 #퍼가디건 #털가디건 #밍크퍼가디건 #블핑 #블랙핑크 #블랙밍크 #밍크가디건 #밍크퍼자켓 #밍크자켓 #진주자켓 #진주가디건 #밴딩자켓 #털자켓 #크롭 #진주 #크롭자켓 #크롭가디건 #크롭퍼자켓 #퍼 #에코 #에코퍼 #에코퍼자켓


In [478]:
hashtag_list[112] = hashtag_list[112][:201]

# #############여기까지만#############

In [ ]:
#table 확인하기
cur.execute(select_command(table))
conn.commit()
cur.fetchall()

In [65]:
#db 연결 종료
conn.cursor().close()
conn.close()

create table yt_clothes (
	id serial,
	title varchar(100),
	date date,
	url varchar(200));